<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/load_with_torchtext_for_que2res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')


In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./simpleR21.csv", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [19]:
print(len(train_dataset))
print(train_dataset[0])
print(train_dataset[0].__dict__.keys())
print(train_dataset[10].query)
print(train_dataset[10].response)

64
dict_keys(['query', 'response'])
['what', 'is', 'your', 'favorite', 'film', '?']
['my', 'favorite', 'file', 'is', 'inception']


In [25]:
QUERY.build_vocab(train_dataset, "glove.6B.100d")
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))

13
khomeini
76


In [0]:
from torchtext.data import BucketIterator

train_iterator = BucketIterator(dataset= train_dataset, batch_size=16
                                ,device=device
                                ,sort_key=lambda x: data.interleave_keys(len(x.query), len(x.response))
                                , repeat = False)

In [8]:
batch = next(iter(train_iterator))
print(batch.__dict__.keys())
print(batch.query.shape)
print(batch.response.shape)
print(batch.input_fields)
print(batch.target_fields)
print(batch.response)

dict_keys(['batch_size', 'dataset', 'fields', 'input_fields', 'target_fields', 'query', 'response'])
torch.Size([6, 16])
torch.Size([12, 16])
['query']
['response']
tensor([[  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2],
        [  4,  13,   4,  12,   4,  16,  91,   4, 106,  13,   4,   4,   4,  27,
           4,   4],
        [ 10,   4,   5, 102,  10,  49,   3,  23,   3,   7,   5,  97,  29,  25,
          14,   8],
        [ 19,  18,  17,  11,  19, 129,   1,  37,   1,  61,  69, 117,  35,  32,
         101,  26],
        [ 18,  19,   3, 109,  38, 105,   1,  55,   1,   4,  56,  48,  46,   7,
           3,  80],
        [ 53,   3,   1,   3,   3, 107,   1,  89,   1,   8,  26,   7,   7,   4,
           1,  39],
        [ 76,   1,   1,   1,   1,   3,   1,   3,   1, 122,  77,  18,  26,   5,
           1,   7],
        [ 71,   1,   1,   1,   1,   1,   1,   1,   1,  18,   3,   9,  64,  81,
           1,  94],
        [ 41,   1,   1,   1,   1,   1,   1,

In [10]:
for batch in (iter(train_iterator)):
  print(batch.query.shape)

torch.Size([7, 16])
torch.Size([12, 16])
torch.Size([7, 16])
torch.Size([6, 16])
